In [ ]:
# user defined R installation
import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #path to your R installation
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

from causaloptim_python import run_experiment, extract_prob_dict



# Load R packages
#igraph = importr('igraph')
causaloptim = importr('causaloptim')
base = importr('base')

#### Data Generation

In [17]:
# Set seed and generate synthetic data in Python
np.random.seed(42)
n = 5000
Z = np.random.binomial(1, 0.5, size=n)
U = np.random.binomial(1, 0.5, size=n)

# X ~ Z + U
logit_X = -1 + 1.5 * Z + 1.2 * U
p_X = 1 / (1 + np.exp(-logit_X))
X = np.random.binomial(1, p_X)

# Y ~ X + U
beta_X, beta_U = 1.0, 1.5
logit_Y = -0.5 + beta_X * X + beta_U * U
p_Y = 1 / (1 + np.exp(-logit_Y))
Y = np.random.binomial(1, p_Y)

df = pd.DataFrame({'Y': Y, 'X': X, 'Z': Z})

# Ground truth ATE
logit_Y1 = -0.5 + beta_X * 1 + beta_U * U
logit_Y0 = -0.5 + beta_X * 0 + beta_U * U
p_Y1 = 1 / (1 + np.exp(-logit_Y1))
p_Y0 = 1 / (1 + np.exp(-logit_Y0))
ATE_true = np.mean(p_Y1 - p_Y0)
print("True ATE:", ATE_true)


True ATE: 0.19877531935424195


#### causaloptim

In [ ]:


graph_str = "(Z -+ X, X -+ Y, Ur -+ X, Ur -+ Y)"
leftside = [1, 0, 0, 0]
latent   = [0, 0, 0, 1]
nvals    = [2, 2, 2, 2]
rlconnect = [0, 0, 0, 0]
monotone = [0, 0, 0, 0]

prob_dict = extract_prob_dict(df)


bounds = run_experiment(graph_str, leftside, latent, nvals, rlconnect, monotone, prob_dict)
print("Bounds:", bounds)

Bounds: (<rpy2.robjects.vectors.FloatVector object at 0x000001762E670910> [14]
R classes: ('numeric',)
[-0.250000], <rpy2.robjects.vectors.FloatVector object at 0x000001762E671890> [14]
R classes: ('numeric',)
[0.450000])


In [13]:
# Create DataFrame
df = pd.DataFrame({'Y': Y, 'X': X, 'Z': Z})

# Compute P(Y, X | Z) safely
joint_counts = df.groupby(['Y', 'X', 'Z']).size().reset_index(name='count')
z_counts = df['Z'].value_counts().to_dict()
joint_counts['p_yx_z'] = joint_counts.apply(
    lambda row: row['count'] / z_counts[row['Z']], axis=1
)
marg_dict = {
    (int(row.Y), int(row.X), int(row.Z)): row.p_yx_z
    for _, row in joint_counts.iterrows()
}

# Helper to fetch probabilities with default fallback
def get_prob(y, x, z):
    return marg_dict.get((y, x, z), 0.0)

# Assign to R using rpy2-compatible scalar format
globalenv["p00_0"] = FloatVector([get_prob(0, 0, 0)])
globalenv["p10_0"] = FloatVector([get_prob(1, 0, 0)])
globalenv["p01_0"] = FloatVector([get_prob(0, 1, 0)])
globalenv["p11_0"] = FloatVector([get_prob(1, 1, 0)])
globalenv["p00_1"] = FloatVector([get_prob(0, 0, 1)])
globalenv["p10_1"] = FloatVector([get_prob(1, 0, 1)])
globalenv["p01_1"] = FloatVector([get_prob(0, 1, 1)])
globalenv["p11_1"] = FloatVector([get_prob(1, 1, 1)])

In [19]:

graph = define_graph(
    graph_str="(Z -+ X, X -+ Y, Ur -+ X, Ur -+ Y)",
    leftside=[1, 0, 0, 0],
    latent=[0, 0, 0, 1],
    nvals=[2, 2, 2, 2],
    rlconnect=[0, 0, 0, 0],
    monotone=[0, 0, 0, 0]
)
graph

<rpy2.robjects.vectors.ListVector object at 0x00000138C5F25FD0> [19]
R classes: ('igraph',)
[FloatSexp..., BoolSexpV..., FloatSexp..., FloatSexp..., ..., NULLType, NULLType, ListSexpV..., SexpEnvir...]
  <no name>: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x00000138C74DA750> [14]
R classes: ('numeric',)
[4.000000]
  <no name>: <class 'rpy2.robjects.vectors.BoolVector'>
  <rpy2.robjects.vectors.BoolVector object at 0x00000138C5F250D0> [10]
R classes: ('logical',)
[       1]
  <no name>: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x00000138C5F0FA50> [14]
R classes: ('numeric',)
[0.000000, 1.000000, 3.000000, 3.000000]
  <no name>: <class 'rpy2.robjects.vectors.FloatVector'>
  <rpy2.robjects.vectors.FloatVector object at 0x00000138C5F0F250> [14]
R classes: ('numeric',)
[1.000000, 2.000000, 1.000000, 2.000000]
...
  <no name>: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x00000138BCC6B450> [0]
  <no name>: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x00000138BCC6B450> [0]
<rpy2.robjects.vectors.ListVector object at 0x00000138C5F25FD0> [19]
R classes: ('igraph',)
[FloatSexp..., BoolSexpV..., FloatSexp..., FloatSexp..., ..., NULLType, NULLType, ListSexpV..., SexpEnvir...]
  <no name>: <class 'rpy2.robjects.environments.Environment'>
  <rpy2.robjects.environments.Environment object at 0x00000138C5F0F750> [4]
R classes: ('environment',)
n items: 4

In [21]:
# R code for causal graph and bounds
r_code = """
library(causaloptim)
library(igraph)



# Analyze causal effect
riskdiff <- "p{Y(X = 1) = 1} - p{Y(X = 0) = 1}"
obj <- analyze_graph(graph, constraints = NULL, effectt = riskdiff)
bounds <- optimize_effect_2(obj)
boundsfunc <- interpret_bounds(bounds = bounds$bounds, obj$parameters)

# Evaluate bounds
ATE_bounds <- boundsfunc(
  p00_0 = p00_0, p00_1 = p00_1,
  p10_0 = p10_0, p10_1 = p10_1,
  p01_0 = p01_0, p01_1 = p01_1,
  p11_0 = p11_0, p11_1 = p11_1
)

ATE_bounds
"""

# Run R code and extract bounds
robjects.r(r_code)
ATE_bounds = robjects.r('ATE_bounds')
print("Bounds from causaloptim:", list(ATE_bounds))

Bounds from causaloptim: [<rpy2.robjects.vectors.FloatVector object at 0x00000138C5F0C6D0> [14]
R classes: ('numeric',)
[-0.159200], <rpy2.robjects.vectors.FloatVector object at 0x00000138C5F0DE50> [14]
R classes: ('numeric',)
[0.519600]]
